In [2]:
pip install langgraph

  Using cached langgraph-0.2.69-py3-none-any.whl.metadata (17 kB)
  Using cached langgraph_checkpoint-2.0.10-py3-none-any.whl.metadata (4.6 kB)
  Using cached langgraph_sdk-0.1.51-py3-none-any.whl.metadata (1.8 kB)
  Using cached msgpack-1.1.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (8.4 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
Using cached langgraph-0.2.69-py3-none-any.whl (148 kB)
Using cached langgraph_checkpoint-2.0.10-py3-none-any.whl (37 kB)
Using cached langgraph_sdk-0.1.51-py3-none-any.whl (44 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
Using cached msgpack-1.1.0-cp312-cp312-macosx_11_0_arm64.whl (82 kB)
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.17.3
    Uninstalling httpcore-0.17.3:
      Successfully uninstalled httpcore-0.17.3
  Attempting uninstall: httpx
    Found existing installatio

# Email Generation Workflow using Javelin and LangGraph

## Overview

This script uses the **Javelin API** and **LangGraph** to perform email generation based on user queries. The flow involves validating the query, refining it, and generating the email.

## Key Components

1. **Javelin API**: 
   - The Javelin API is used to validate and refine the user's query by providing a route (`testing` in this case). It helps assess whether the request is suitable for email generation and aids in refining the query for clarity.
   - **Headers**: Contains the API key and the route for Javelin (`x-javelin-route`).
   
2. **LangGraph**:
   - LangGraph is used to manage the flow of agents, where each agent is responsible for a specific task: 
     - **Agent 1**: Validates if the query can be turned into an email.
     - **Agent 2**: Refines the query.
     - **Agent 3**: Generates the email from the refined query.
   
3. **StateGraph**:
   - LangGraph’s `StateGraph` is used to create a flowchart-like structure for the process. It connects the agents in a sequence, ensuring that the right steps are followed in the correct order.

## Workflow

- **Step 1**: The user's query is first passed to **Agent 1**, where it's validated by querying Javelin. If valid, it moves to **Agent 2**.
- **Step 2**: **Agent 2** refines the query for clarity, ensuring that the purpose and recipients are clear.
- **Step 3**: **Agent 3** takes the refined query and generates an email.
- The system continues its flow until the final email is generated.

## How Javelin Plays a Role

Javelin is used as a "router" for query validation and refinement:
- It provides the necessary decision-making process by analyzing the query and returning whether it's suitable for email generation.
- The API returns a JSON response that determines the flow (valid/invalid).

## LangGraph’s Role

LangGraph is responsible for managing the execution flow:
- It ensures the agents perform their tasks in the correct order.
- It allows for dynamic branching, ensuring that if the query is not valid, the process stops.

## Final Goal

The final goal is to:
1. Validate the user’s query.
2. Refine it if necessary.
3. Generate a valid email based on the refined query.

Each of these steps is achieved through the seamless interaction between Javelin (for validation) and LangGraph (for orchestrating the flow of tasks).


In [10]:
from openai import OpenAI
import os
import json
from typing import List, TypedDict, Literal
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv
load_dotenv()
# -------------------------------
# Initialize Clients using Unified Endpoint
# -------------------------------
# Set your API keys in your environment variables: OPENAI_API_KEY and JAVELIN_API_KEY
openai_api_key = os.getenv("OPENAI_API_KEY")
javelin_api_key = os.getenv("JAVELIN_API_KEY")

# Create a plain OpenAI client
openai_client = OpenAI(api_key=openai_api_key)

# Initialize Javelin unified endpoint client
from javelin_sdk import JavelinClient, JavelinConfig

config = JavelinConfig(javelin_api_key=javelin_api_key)
client = JavelinClient(config)
# Register the OpenAI client with the unified route name (e.g., "openai_univ")
client.register_openai(openai_client, route_name="openai_univ")

# -------------------------------
# Define Message Structure
# -------------------------------
class MessagesState(TypedDict):
    messages: List[dict]
    valid: bool            # Indicates if it's a valid email request
    refined_query: str     # Stores the refined query (if applicable)

# -------------------------------
# Agent Functions Using Unified Endpoint
# -------------------------------
def agent_1(state: MessagesState) -> MessagesState:
    messages = state["messages"]
    user_message = messages[-1]["content"].lower()

    # Validate if the query is valid for email generation
    completion = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": (
                f"Analyze the following user request: '{user_message}'. "
                "Determine if it's valid for generating an email based on the request. "
                "The request is valid if it specifies who to write to, why the email is being written, "
                "and what to include in the email. If the request does not pertain to generating an email, "
                "return false for validity. Please justify the validity status in the response. "
                "Return a JSON object with 'valid' (true/false), 'response', and 'extracted_info' "
                "with details such as recipient and reason."
            )}
        ]
    )
    
    response_json = completion.choices[0].message.content.strip()
    try:
        parsed_response = json.loads(response_json)
        valid = parsed_response.get("valid", False)
    except json.JSONDecodeError:
        valid = False

    return {
        "messages": messages,
        "valid": valid,
        "refined_query": ""
    }

# Agent 2: Refine the query if valid
def agent_2(state: MessagesState) -> MessagesState:
    if not state["valid"]:
        return state  # Skip if invalid

    messages = state["messages"]
    user_message = messages[-1]["content"]

    # Refine the query if valid
    completion = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": (
                "You are a helpful assistant to refine the query and highlight the points."
            )},
            {"role": "user", "content": (
                f"Refine the following query for email generation: '{user_message}'. "
                "Ensure to highlight who to send the email to, why, and what needs to be included. "
                "Important: do not create an email. Just return a refined user query."
            )}
        ]
    )

    refined_query = completion.choices[0].message.content.strip()

    return {
        "messages": messages,
        "valid": True,
        "refined_query": refined_query
    }

def agent_3(state: MessagesState) -> MessagesState:
    if not state["refined_query"]:
        return state

    refined_query = state["refined_query"]

    # Generate the email based on the refined query
    completion = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Generate an email based on the following details: '{refined_query}'"}
        ]
    )

    email_content = completion.choices[0].message.content.strip()

    return {
        "messages": state["messages"] + [{"role": "assistant", "content": email_content}],
        "valid": True,
        "refined_query": refined_query
    }

# Decision function to determine flow
def should_continue(state: MessagesState) -> Literal["tools", END]:
    if state["valid"]:
        if not state["refined_query"]:
            print("should_continue decision: tools (Proceeding to query refinement)")
            return "tools"
        else:
            print("should_continue decision: tools (Proceeding to email generation)")
            return "tools"
    else:
        print("should_continue decision: __end__ (Stopping, invalid request)")
        return END

# -------------------------------
# Set Up the State Machine
# -------------------------------
graph = StateGraph(MessagesState)

# Define the nodes for each agent
graph.add_node("agent_1", agent_1)
graph.add_node("agent_2", agent_2)
graph.add_node("agent_3", agent_3)

# Define the flow of agents
graph.add_edge("agent_1", "agent_2")  # Proceed to agent 2 if agent 1 validates
graph.add_edge("agent_2", "agent_3")  # Proceed to agent 3 if agent 2 refines the query

# Set the entry point for the state machine
graph.set_entry_point("agent_1")

# Compile the graph
app = graph.compile()

# Test cases
queries = [
    "I want to send an email to my boss to request more leave because I am feeling unwell.",
    "Who is the President of India?"
]

for query in queries:
    print("\nUser Query:", query)
    initial_state = {
        "messages": [{"role": "user", "content": query}],
        "valid": True,
        "refined_query": ""
    }

    final_state = app.invoke(initial_state, config={"debug": True})
    assistant_response = next(
        (msg["content"] for msg in final_state["messages"] if msg["role"] == "assistant"),
        "No response."
    )
    print("Final Response:", assistant_response)



User Query: I want to send an email to my boss to request more leave because I am feeling unwell.
Final Response: Subject: Request for Additional Leave Due to Illness

Dear [Boss's Name],

I am writing to request additional leave from work due to feeling unwell. I have been experiencing worsening symptoms that have made it difficult to fully recover within the initially scheduled time off. I am reaching out to seek your understanding and support during this time.

I believe that taking additional time off to focus on my health will allow me to recuperate fully and return to work with renewed energy and focus. I have been in touch with my healthcare provider and am following their guidance to ensure a swift recovery.

I understand the importance of my role and the impact of my absence on the team. I am committed to staying updated on any urgent matters and ensuring a smooth transition of responsibilities during my extended leave.

I would appreciate your guidance on the process for req